In [1]:
import pandas as pd
comments=pd.read_excel('evaluations_overall_comments.xlsx')

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

c:\Users\feryo\OneDrive\Documents\GitHub\FYP-GoodHealth\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MODEL STRAIGHT FROM HUGGINGFACE

In [32]:
# download label mapping
# labels=[]
# mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
# with urllib.request.urlopen(mapping_link) as f:
#     html = f.read().decode('utf-8').split("\n")
#     csvreader = csv.reader(html, delimiter='\t')
# labels = [row[1] for row in csvreader if len(row) > 1]

# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

In [3]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
comments['encoded']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output']=comments['encoded'].apply(lambda x:model(**x))
comments['scores']=comments['output'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

         Target:                      Evaluator:       Rotation Dates:  \
0     Eliza Chen      Dr. Ong, Andrew Ming Liang    7/1/2022-7/31/2022   
1     Eliza Chen         Dr. Chan, Webber Pak Wo    7/1/2022-7/31/2022   
2     Eliza Chen              Dr. Lim, Chee Hooi    7/1/2022-7/31/2022   
3     Eliza Chen         Dr. Chang, Jason Pik Eu    8/1/2022-8/31/2022   
4     Eliza Chen              Dr. Tan, Chee Kiat    8/1/2022-8/31/2022   
..           ...                             ...                   ...   
163  Rachel Yeap     Dr. Tan, Malcolm Teck Kiang    7/1/2022-7/31/2022   
164  Rachel Yeap  Dr. Khor, Christopher Jen Lock    8/1/2022-8/31/2022   
165  Rachel Yeap               Dr. Liou, Wei Lun    8/1/2022-8/31/2022   
166  Rachel Yeap             Dr. Loo, Khang Ning    9/1/2022-10/2/2022   
167  Rachel Yeap        Dr. Kwek, Andrew Boon Eu  10/3/2022-10/31/2022   

                         Service:  \
0                             - -   
1       SHS-GASTRO:BASIC ENDO-SGH   


In [5]:
# NEUTRAL COMMENTS: 
comments['neu']=comments['scores'].apply(lambda x: x[1])
# comments[['Answer:','neu']].loc[comments['neu']>0.7]
# NEGATIVE COMMENTS
comments['neg']=comments['scores'].apply(lambda x: x[0])
# comments[['Answer:','neg']].loc[comments['neg']>0.2].values.tolist()
# NON POSITIVE COMMENTS
comments['pos']=comments['scores'].apply(lambda x: x[2])
# comments[['Answer:','pos']].loc[comments['pos']<0.2].values.tolist()

MODEL TRAINED ON IMDB DATASET (bad)

In [ ]:
MODEL = f"trained-model"

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['processed2']=comments['Answer:'].apply(lambda x: preprocess(x))
comments['encoded2']=comments['processed2'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output2']=comments['encoded2'].apply(lambda x:model(**x))
comments['scores2']=comments['output2'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

In [ ]:
comments['scores3']=comments['scores2'].apply(lambda x:x[1])
comments['scores3'].values.tolist()

MODEL TRAINED ON SELF LABELLED DATASET 

In [8]:
MODEL = f"csv_batch16"

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['encoded3']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output3']=comments['encoded3'].apply(lambda x:model(**x))
comments['c16']=comments['output3'].apply(lambda x:softmax(x[0][0].detach().numpy()))
# NEUTRAL COMMENTS: 
comments['neu c16']=comments['c16'].apply(lambda x:x[1])
comments['neg c16']=comments['c16'].apply(lambda x:x[0])
comments['pos c16']=comments['c16'].apply(lambda x: x[2])


In [17]:
MODEL = f"finetuned-model"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['encoded10']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output10']=comments['encoded10'].apply(lambda x:model(**x))
comments['c10']=comments['output10'].apply(lambda x:softmax(x[0][0].detach().numpy()))
# NEUTRAL COMMENTS: 
comments['neu c10']=comments['c10'].apply(lambda x:x[1])
comments['neg c10']=comments['c10'].apply(lambda x:x[0])
comments['pos c10']=2


In [18]:
print(comments[['pos','neu','neg','pos c10','neu c10','neg c10']])

          pos       neu       neg  pos c10   neu c10   neg c10
0    0.770696  0.216474  0.012830        2  0.395472  0.159170
1    0.937809  0.059670  0.002520        2  0.074460  0.013259
2    0.078489  0.744895  0.176617        2  0.786172  0.144544
3    0.876909  0.112595  0.010496        2  0.309187  0.158402
4    0.198578  0.611597  0.189825        2  0.451928  0.451729
..        ...       ...       ...      ...       ...       ...
163  0.397909  0.526805  0.075286        2  0.467569  0.404326
164  0.974589  0.022789  0.002622        2  0.021468  0.005085
165  0.926716  0.069175  0.004109        2  0.119251  0.016472
166  0.806028  0.183774  0.010198        2  0.223648  0.066089
167  0.732754  0.252707  0.014539        2  0.407553  0.346090

[168 rows x 6 columns]


In [15]:
MODEL = f"csv_batch10"

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['encoded10']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output10']=comments['encoded10'].apply(lambda x:model(**x))
comments['c10']=comments['output10'].apply(lambda x:softmax(x[0][0].detach().numpy()))
# NEUTRAL COMMENTS: 
comments['neu c10']=comments['c10'].apply(lambda x:x[1])
comments['neg c10']=comments['c10'].apply(lambda x:x[0])
comments['pos c10']=comments['c10'].apply(lambda x: x[2])

In [10]:
MODEL = f"csv_batch8"

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['encoded4']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output4']=comments['encoded4'].apply(lambda x:model(**x))
comments['c8']=comments['output4'].apply(lambda x:softmax(x[0][0].detach().numpy()))
# NEUTRAL COMMENTS: 
comments['neu c8']=comments['c8'].apply(lambda x:x[1])
comments['neg c8']=comments['c8'].apply(lambda x:x[0])
comments['pos c8']=comments['c8'].apply(lambda x: x[2])

In [17]:
MODEL = f"excel_batch16"

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

comments['encoded5']=comments['Answer:'].apply(lambda x:tokenizer(x,return_tensors='pt'))
comments['output5']=comments['encoded5'].apply(lambda x:model(**x))
comments['x16']=comments['output5'].apply(lambda x:softmax(x[0][0].detach().numpy()))
print(comments)

comments['neu x16']=comments['x16'].apply(lambda x:x[1])
comments['neg x16']=comments['x16'].apply(lambda x:x[0])
comments['pos x16']=comments['x16'].apply(lambda x: x[2])

KeyboardInterrupt: 

In [16]:
comments.to_excel('comments_with_score2.xlsx')

